In [1]:
import torch
import torch.nn as nn
import torchsummary as summary

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
X = torch.tensor([
    [1, 2],
    [3, 4],
    [5, 6],
    [7, 8],
])

y = torch.tensor([
    [3],
    [7],
    [11],
    [15],
])

In [4]:
X, y

(tensor([[1, 2],
         [3, 4],
         [5, 6],
         [7, 8]]),
 tensor([[ 3],
         [ 7],
         [11],
         [15]]))

In [5]:
print(f'type:\nX: {X.dtype} | y: {y.dtype}')

type:
X: torch.int64 | y: torch.int64


In [6]:
X_float = X.type(torch.float32)
y_float = y.type(torch.float32)

In [7]:
X_float, y_float

(tensor([[1., 2.],
         [3., 4.],
         [5., 6.],
         [7., 8.]]),
 tensor([[ 3.],
         [ 7.],
         [11.],
         [15.]]))

In [8]:
print(f'type:\nX: {X_float.dtype} | y: {y_float.dtype}')

type:
X: torch.float32 | y: torch.float32


In [9]:
X_device = X_float.to(device)
y_device = y_float.to(device)

In [10]:
X_device, y_device

(tensor([[1., 2.],
         [3., 4.],
         [5., 6.],
         [7., 8.]], device='cuda:0'),
 tensor([[ 3.],
         [ 7.],
         [11.],
         [15.]], device='cuda:0'))

In [11]:
layer= nn.Linear(in_features=2, out_features=3, device=device)(X_device)
layer

tensor([[-0.6797,  0.1070,  1.6480],
        [-1.4176,  0.9636,  4.1210],
        [-2.1554,  1.8201,  6.5939],
        [-2.8932,  2.6767,  9.0669]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [57]:
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense_1 = nn.Linear(in_features=2, out_features=16)
        self.relu_1 = nn.ReLU()
        self.dense_2 = nn.Linear(in_features=16, out_features=1)

    def forward(self, x):
        x = self.dense_1(x)
        x = self.relu_1(x)
        x = self.dense_2(x)
        
        return x
    
simpleNN = SimpleNN().to(device)

In [58]:
summary.summary(simpleNN, torch.zeros(1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 16]                  48
├─ReLU: 1-2                              [-1, 16]                  --
├─Linear: 1-3                            [-1, 1]                   17
Total params: 65
Trainable params: 65
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 16]                  48
├─ReLU: 1-2                              [-1, 16]                  --
├─Linear: 1-3                            [-1, 1]                   17
Total params: 65
Trainable params: 65
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [60]:
LOSS = nn.MSELoss()
OPTIMIZER = torch.optim.Adam(simpleNN.parameters(), lr=1e-3)

In [14]:
y_loss = simpleNN(X_device)
loss_value = LOSS(y_loss, y_device)
print(loss_value)

tensor(149.7222, device='cuda:0', grad_fn=<MseLossBackward0>)


In [23]:
loss_history = []
epochs = 200

for epoch in range(1, epochs + 1):
    OPTIMIZER.zero_grad()
    
    y_outputs = simpleNN(X_device)
    loss_value = LOSS(y_outputs, y_device)
    loss_value.backward()
    
    OPTIMIZER.step()

    loss_history.append(loss_value.item())

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss_value}')

Epoch 100, Loss: 74.03160095214844
Epoch 200, Loss: 27.51620101928711


In [168]:
n_data = 900
step_x1 = 4
step_x2 = 6
start_x1 = 2
start_x2 = 10

# Menghitung jumlah elemen agar kedua tensor memiliki panjang yang sama
len_x1 = (n_data - start_x1) // step_x1
len_x2 = (n_data - start_x2) // step_x2
min_len = min(len_x1, len_x2)

# Membuat tensor X dan Y dengan panjang yang sama
X_1 = torch.arange(start_x1, start_x1 + min_len * step_x1, step_x1).reshape(-1, 1)
X_2 = torch.arange(start_x2, start_x2 + min_len * step_x2, step_x2).reshape(-1, 1)

Y = X_1 + 2 * X_2 

# Menggabungkan kedua tensor
X = torch.concat((X_1, X_2), dim=1).type(torch.float32)
print(X.shape, Y.shape)
print(X[:5], Y[:5])

torch.Size([148, 2]) torch.Size([148, 1])
tensor([[ 2., 10.],
        [ 6., 16.],
        [10., 22.],
        [14., 28.],
        [18., 34.]]) tensor([[22],
        [38],
        [54],
        [70],
        [86]])


In [169]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __getitem__(self, index):
        
        x_float = self.x.type(torch.float32)
        y_float = self.y.type(torch.float32)

        x_clone = x_float.clone().detach()
        y_clone = y_float.clone().detach()

        X_grad = x_clone.requires_grad_(True)
        y_grad = y_clone.requires_grad_(True)

        return X_grad[index].to(device), y_grad[index].to(device)
    
    def __len__(self):
        return len(self.x)

torch_data = MyDataset(X, Y)

In [170]:
torch_data.__len__()

148

In [171]:
for feature, label in torch_data:
    print(feature, label)

tensor([ 2., 10.], device='cuda:0', grad_fn=<ToCopyBackward0>) tensor([22.], device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([ 6., 16.], device='cuda:0', grad_fn=<ToCopyBackward0>) tensor([38.], device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([10., 22.], device='cuda:0', grad_fn=<ToCopyBackward0>) tensor([54.], device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([14., 28.], device='cuda:0', grad_fn=<ToCopyBackward0>) tensor([70.], device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([18., 34.], device='cuda:0', grad_fn=<ToCopyBackward0>) tensor([86.], device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([22., 40.], device='cuda:0', grad_fn=<ToCopyBackward0>) tensor([102.], device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([26., 46.], device='cuda:0', grad_fn=<ToCopyBackward0>) tensor([118.], device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([30., 52.], device='cuda:0', grad_fn=<ToCopyBackward0>) tensor([134.], device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([34., 58.], device='cu

In [172]:
train_split = int(len(torch_data) * .9)
valid_split = len(torch_data) - int(len(torch_data) * .9)

print(f'total: {len(torch_data)}')
print(f'train total: {train_split}')
print(f'valid total: {valid_split}')
print(f'total train + valid: {train_split + valid_split}')

total: 148
train total: 133
valid total: 15
total train + valid: 148


In [173]:
train_set, val_set = torch.utils.data.random_split(torch_data, [train_split, valid_split])

In [174]:
len(train_set), len(val_set)

(133, 15)

In [176]:
for feature, label in train_set:
    print(feature, label)
    break

tensor([306., 466.], device='cuda:0', grad_fn=<ToCopyBackward0>) tensor([1238.], device='cuda:0', grad_fn=<ToCopyBackward0>)


In [134]:
torch_data_train_pipeline = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=False)

torch_data_valid_pipeline = torch.utils.data.DataLoader(val_set, batch_size=4, shuffle=False)

In [140]:
for feature, label in torch_data_train_pipeline:
    print(feature)
    break

tensor([[ 4758.,  7144.],
        [ 1878.,  2824.],
        [18906., 28366.],
        [ 7966., 11956.]], device='cuda:0', grad_fn=<StackBackward0>)


In [61]:
loss_history = []
epochs = 10

for epoch in range(1, epochs + 1):
    for batch, (feature, label) in enumerate(torch_data_pipeline, 1):
        OPTIMIZER.zero_grad()
        
        y_outputs = simpleNN(X_device)
        loss_value = LOSS(y_outputs, y_device)
        loss_value.backward()
        
        OPTIMIZER.step()

        loss_history.append(loss_value.item())

        if batch % len(torch_data_pipeline) == 0:
            print(f'Epoch {epoch}, Loss: {loss_value}')

Epoch 1, Loss: 18.223663330078125
Epoch 2, Loss: 0.13342314958572388
Epoch 3, Loss: 0.000689713517203927
Epoch 4, Loss: 0.000592874304857105
Epoch 5, Loss: 0.0005092871142551303
Epoch 6, Loss: 0.0004273323866073042
Epoch 7, Loss: 0.0003505703352857381
Epoch 8, Loss: 0.0002816477499436587
Epoch 9, Loss: 0.00022221647668629885
Epoch 10, Loss: 0.00017310766270384192
